In [2]:
from fuzzywuzzy import fuzz
import pandas as pd 

In [3]:
ent_df = pd.read_csv('entity.csv')
ent_df.head()

,DB_ID,Name,Blck_List
0,14173,ALVAREZ CARLOS,0
1,14174,TARANTOLA SALVATORE,0
2,14175,HOME DEVCO TIVOLI ISLES LLC,1
3,14176,LEVIN MICHAEL,0
4,14177,ROSS NICHOLAS J,0


This code was used to find most commont word tokens.  
I used this to build an ignore list where I am filtering out common  
words to describe a business. IE  Corp, Inc, Co

```python
from collections import Counter
ent_counter = Counter()
for names in ent_df['Name'].str.split():
    ent_counter.update(names)
with open('common.csv', mode='w') as f:
    print("Name, count", file=f)
    for c in  ent_counter.most_common(100):
        print(f'{c[0]},{c[1]}', file=f)
```

In [4]:
ignore = pd.read_csv('common.csv')
ignore_list = ignore['Name'].to_list()

In [5]:
def clean(e):
    return all([t not in ignore_list for t in e.split()])

ent_df_clean = ent_df[ent_df['Name'].map(clean)].copy()
# creating a new pf object not ref

In [6]:
print(f'counts: original -> {len(ent_df)} cleaned -> {len(ent_df_clean)}')

counts: original -> 17187 cleaned -> 13865


In [7]:
ent_df_clean['token_count'] = ent_df_clean['Name'].str.split().str.len()
ent_df_clean.head(10)

,DB_ID,Name,Blck_List,token_count
0,14173,ALVAREZ CARLOS,0,2
1,14174,TARANTOLA SALVATORE,0,2
3,14176,LEVIN MICHAEL,0,2
4,14177,ROSS NICHOLAS J,0,3
5,14178,PISCATELLI ALAN,0,2
6,14179,SCHNEIDER ROBERT,0,2
7,14180,FOBBS RAMONA A,0,3
8,14181,"KELLER FRED ESTATE,SHEPHERD DANIEL J PERSONAL ...",1,7
9,14182,"SHEPHERD DANIEL J PERSONAL REPRESENTATIVE,KELL...",1,7
10,14183,MICHEL JEANTRAL,0,2


In [8]:
groups = ent_df_clean.groupby('token_count').groups

In [9]:
list(groups.items())[0]

(1, Int64Index([ 1418,  2182,  2927,  3868,  5105,  9648,  9732,  9968, 10038,
             10275, 10454, 10976, 11365, 11419, 11690, 11738, 11754, 11780,
             11793, 11959, 12123, 13084, 13921, 14297, 14299, 14314, 14341,
             14477, 14690, 14730, 15020, 15294, 17117],
            dtype='int64'))

In [10]:
ent_df_clean.loc[2182]

DB_ID               16355
Name           BANKUNITED
Blck_List               1
token_count             1
Name: 2182, dtype: object

In [18]:
import whoswho

ModuleNotFoundError: No module named 'whoswho'

In [14]:

def test_methods(s1, s2):
    simple_ratio = fuzz.ratio(s1, s2)
    partial_ratio = fuzz.partial_ratio(s1, s2)
    token_sort_ratio = fuzz.token_sort_ratio(s1, s2)
    token_set_ratio = fuzz.token_set_ratio(s1, s2)
    print(f'simple: {simple_ratio} partial: {partial_ratio} token_sort: {token_sort_ratio}, token_set: {token_set_ratio}')

In [15]:
#same tokens simple mispelling
test_methods('BARNES JAMES', 'BARNS JAMES')

simple: 96 partial: 91 token_sort: 96, token_set: 96


In [17]:
#same tokens different spelling (nicknames/shorthand)
test_methods('Albert Zuniga', 'Alberto Zuniga')
test_methods('Robert Meuller', 'Bill Meuller')

simple: 96 partial: 92 token_sort: 96, token_set: 96
simple: 62 partial: 67 token_sort: 54, token_set: 74
